## Introduction to SciBmad  

Differentiable simulation codes are necessary tools in accelerator physics to properly compute (via integration) Taylor maps representing the particle transport, and subsequently extract the lattice functions from. Early examples of differentiable accelerator physics codes are the Polymorphic Tracking Code (PTC) and COSY-INFINITY, both developed in the early 1990s. Modern examples of differentiable codes are CERN's `MAD-NG`, and Cornell's `SciBmad`:

`SciBmad` is a new, modular, polymorphic, differentiable, and portable (CPU/GPU compatible) accelerator physics code being developed at Cornell. This Jupyter notebook will show some its basic features, including lazily evaluated deferred expressions, polymorphism, and differentiability.

In [ ]:
# This only needs to be done once
import Pkg; 
Pkg.add("SciBmad")
Pkg.add("GLMakie") # Other good plotting packages are CairoMakie, Plots, WGLMakie

In [1]:
using SciBmad

Let's first make a FODO cell consisting of a quadrupole, sextupole, bend, and drift. This uses the [`Beamlines.jl`](https://github.com/bmad-sim/Beamlines.jl) package within the SciBmad family, which is automatically included when `using SciBmad`. We require by design that the focusing quadrupole strength is negative that of the defocusing quadrupole strength, and likewise for the sextupole. To achieve this, we will use `SciBmad`'s deferred expressions

In [2]:
# Make a FODO cell requiring that
# qf.Kn1 = -qd.Kn1 
# and
# sf.Kn2 = -sd.Kn2
# This is done using a "deferred expression"
# to form a "closure" of the variables we define:

K1 = 0.36;
K2 = 0.1;

qf = Quadrupole(Kn1=DefExpr(() -> +K1), L=0.5);
sf = Sextupole(Kn2=DefExpr(() -> +K2), L=0.2);
d  = Drift(L=0.6);
b  = SBend(L=6.0, angle=pi/132);
qd = Quadrupole(Kn1=DefExpr(() -> -K1), L=0.5);
sd = Sextupole(Kn2=DefExpr(() -> -K2), L=0.2);


fodo_line = [qf, sf, d, b, d, qd, sd, d, b, d];
fodo = Beamline(fodo_line, species_ref=Species("electron"), E_ref=18e9);

Now if we update `K1`, both `qf` and `qd` will be (lazily) updated.

In [3]:
K1 = 0.3
println(qf.Kn1)
println(qd.Kn1)

0.3
-0.3


We can get the Twiss parameters using `twiss` 

In [4]:
t = twiss(fodo)

plt = plot()
plot!(plt, t.s, t.beta_1, label="beta_1")
plot!(plt, t.s, t.beta_2, label="beta_2")
xlabel!(plt, "s [m]")
ylabel!(plt, "Beta Function [m]")
display(plt)
t

LoadError: UndefVarError: `plot` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Let's compute the transport matrix for this FODO cell using automatic differentiation (AD). Because it is a periodic (closed) geometry, need to expand around the closed orbit. The closed orbit is computed in the `twiss` call

In [5]:
co = [t.orbit_x[1], t.orbit_px[1], t.orbit_y[1], t.orbit_py[1], t.orbit_z[1], t.orbit_pz[1]]

6-element Vector{Float64}:
  2.5673907444456746e-17
 -3.469446951953614e-18
  1.9503102460948876e-33
  2.5494277617600177e-34
  3.552713678800501e-15
  0.0

We see that in this case the closed orbit is the "zero orbit", which makes sense because all the magnets are perfectly aligned and there are no RF cavities. Note that in general the "zero orbit" is not always the closed orbit (in the case of a periodic accelerator) or the reference orbit (in the case of a linac). We must always expand around some real trajectory.

Now we will use [`GTPSA.jl`](https://github.com/bmad-sim/GTPSA.jl) to compute the Taylor map, to 2nd order. `GTPSA.jl` is a full-featured interface to the Generalised Truncated Power Series Algebra library, written by Laurent Deniau for CERN's `MAD-NG` code, and is another package in the `SciBmad` family. Because `SciBmad` is polymorphic, all we need to do is track a single particle on the closed orbit where the numbers are Truncated Power Series (`TPS`) types instead of the regular 64-bit floats `Float64`s.

First, we define a `Descriptor` to define the number of variables (6) and the truncation order of our Taylor series (2).

In [6]:
D2 = Descriptor(6, 2) # 6 variables, 2nd order

Descriptor(NV=6, MO=2)

We will then get our special `TPS` number types. Specifically, we will start with the "identity map", or a vector of unit "wiggles". E.g., wiggle the input a bit, see how the output wiggles

In [7]:
# Then get a vector of `TPS`s representing an identity of the variables:
dv = @vars(D2) # Think of these as unit "wiggles"

 # Finally initialize our particle coordinates
v0 = zeros(6) # Closed orbit
v = v0 + dv   # Particle as a TPSA (wiggle around the closed orbit)

6-element Vector{TPS64{Descriptor(NV=6, MO=2)}}:
 Index Coefficient                Order   Exponent
--------------------------------------------------------------------
   1:   1.0000000000000000e+00      1      1   0   0   0   0   0
--------------------------------------------------------------------
   2:   1.0000000000000000e+00      1      0   1   0   0   0   0
--------------------------------------------------------------------
   3:   1.0000000000000000e+00      1      0   0   1   0   0   0
--------------------------------------------------------------------
   4:   1.0000000000000000e+00      1      0   0   0   1   0   0
--------------------------------------------------------------------
   5:   1.0000000000000000e+00      1      0   0   0   0   1   0
--------------------------------------------------------------------
   6:   1.0000000000000000e+00      1      0   0   0   0   0   1


Now we track!

In [8]:
b0 = Bunch(v)
track!(b0, fodo)  # b0 is mutated in-place with the result
println(b0.coords.v)

[ Info: Setting bunch.species = Species("electron", -1 e, 0.51099895069 MeV c^-2, 0.5 h_bar, -9.2847646917e-24 J T^-1, 0.0, AtomicAndPhysicalConstants.Kind.LEPTON) (reference species from the Beamline)
[ Info: Setting bunch.R_ref = -60.04153711147289 (reference R_ref from the Beamline)


6-element Matrix{TPS64{Descriptor(NV=6, MO=2)}}:
 Index Coefficient                Order   Exponent
--------------------------------------------------------------------
   1:   2.5673907444456746e-17      0      0   0   0   0   0   0
   1:  -1.5014406748569284e+00      1      1   0   0   0   0   0
   1:   2.4337323961276095e+01      1      0   1   0   0   0   0
   1:   3.0902218513873479e-16      1      0   0   1   0   0   0
   1:   1.1862399218856096e-15      1      0   0   0   1   0   0
   1:   4.6559967240029487e-01      1      0   0   0   0   0   1
   1:  -2.1624629039941584e-01      2      2   0   0   0   0   0
   1:  -5.9739097567507493e-01      2      1   1   0   0   0   0
   1:   6.0045879718470401e+00      2      0   2   0   0   0   0
   1:   2.0259254391159418e-17      2      1   0   1   0   0   0
   1:   3.6346854686241299e-17      2      0   1   1   0   0   0
   1:  -7.3954551315820696e-02      2      0   0   2   0   0   0
   1:   9.5315805435515612e-18      2      1   0   

This is the real nonlinear Taylor map, computed from the integrator itself - no matrix or map is stored internally.

The exponent specifies the power of each variable, e.g. the last row in `1: ...` corresponds to $\delta^2$ - it defines how $x$ depends on $\delta^2$.

We can also get the linear part (as a matrix):

In [9]:
# We can clean up the output with
GTPSA.show_eps = 1e-10

# Also print the matrix (linear part)
GTPSA.jacobian(b0.coords.v)

6×6 Matrix{Float64}:
 -1.50144      24.3373        3.09022e-16   1.18624e-15  0.0   0.4656
 -0.173973      2.15396       4.0393e-17    1.55079e-16  0.0   0.0614828
 -2.39273e-17   1.10387e-15   0.868579      6.81317      0.0   1.28673e-17
 -3.13516e-18   1.44346e-16  -0.173989     -0.213474     0.0   1.68319e-18
  0.0113111    -0.493447     -3.89334e-18  -1.49434e-17  1.0  -0.00694548
  0.0           0.0           0.0           0.0          0.0   1.0

To get the tunes, we need to go to the normal form, where the motion lies on circles in phase space with tunes as functions of solely the radius of the circle (amplitude-dependent rotations). E.g. we seek a nonlinear map $\vec{a}$ such that $\vec{r} = \vec{a}^{-1}\circ\vec{m}\circ\vec{a}$ where $\vec{m}$ is the one-turn map. $\vec{r}$ will then, being in circle world, have the form

$$
\begin{equation}
    \begin{pmatrix}
        \bar{q}_1 \\ \bar{p}_1 \\ \vdots \\ \bar{q}_n \\ \bar{p}_n
    \end{pmatrix}_{t_0+T} = 
    \underbrace{\begin{pmatrix}
        \mathsf{R}(\mu_i(\vec{J})) & \mathsf{0} & \mathsf{0} \\
        \mathsf{0} & \ddots & \mathsf{0}\\
        \mathsf{0} & \mathsf{0} & \mathsf{R}(\mu_n(\vec{J}))
    \end{pmatrix}}_{\let\scriptstyle\textstyle\substack{\vec{r}}}
\begin{pmatrix}
        \bar{q}_1 \\ \bar{p}_1 \\ \vdots \\ \bar{q}_n \\ \bar{p}_n
    \end{pmatrix}_{t_0},\ \  \mathsf{R}(\alpha)= \begin{pmatrix}
        \cos{\alpha} && \sin{\alpha} \\
        -\sin{\alpha} && \cos{\alpha}
    \end{pmatrix} \ .
\end{equation}
$$

where $(\bar{q}_1,\bar{p}_1) = (\sqrt{2J_1}\cos{\phi_1},\sqrt{2J_1}\sin{\phi_1})$.

`SciBmad`'s `NonlinearNormalForm.jl` package makes it very easy to compute `a`, with a single function `normal`. We will elevate the tracking map result to a bona-fide `DAMap` (differential algebraic map) type, and then compute `a`:

In [10]:
m = DAMap(v=b0.coords.v)
a = normal(m)

DAMap{StaticArraysCore.MVector{5, Float64}, StaticArraysCore.MVector{6, TPS64{Descriptor(NV=6, MO=2)}}, Nothing, Nothing}([0.0, 0.0, 0.0, 0.0, 0.0], 6-element StaticArraysCore.MVector{6, TPS64{Descriptor(NV=6, MO=2)}}:
 Index Coefficient                Order   Exponent
--------------------------------------------------------------------
   1:  -5.0740641298145208e+00      1      0   1   0   0   0   0
   1:   7.1173080782844123e-01      1      0   0   0   0   0   1
   1:  -3.2440195507262537e+00      2      2   0   0   0   0   0
   1:  -1.0588609141558833e-01      2      1   1   0   0   0   0
   1:  -6.2957227019419260e-01      2      0   2   0   0   0   0
   1:  -7.7843206796179210e-01      2      0   0   2   0   0   0
   1:   2.0172973343682132e-01      2      0   0   1   1   0   0
   1:  -9.4879578008434196e-01      2      0   0   0   2   0   0
   1:   4.0762360205410986e-01      2      1   0   0   0   0   1
   1:   2.2525273815312095e+00      2      0   1   0   0   0   1
   1:   4.8

We now can compute the nonlinear rotation $\vec{r}$:

In [11]:
ai = inv(a)
r = ai * m * a
GTPSA.jacobian(r.v)

6×6 Matrix{Float64}:
  0.326257      0.945281     -2.38455e-16   7.58989e-17  0.0   1.38778e-16
 -0.945281      0.326257     -1.2326e-32   -1.84889e-32  0.0   4.16334e-17
 -3.78653e-29  -6.31089e-30   0.327553      0.944833     0.0  -1.26218e-29
  0.0           0.0          -0.944833      0.327553     0.0   1.57772e-30
 -4.71845e-16  -3.33501e-16   0.0           2.46519e-32  1.0  -0.0255521
  0.0           0.0           0.0           0.0          0.0   1.0

It is easiest to view this in the complex phasors basis where the coordinates are $(\tilde{q}_i,\tilde{p}_i)=(\sqrt{J_i}e^{+\textrm{i}\phi_i},\sqrt{J_i}e^{-\textrm{i}\phi_i})$. That way, we see explicitly that the monomials $(\tilde{q}_i\tilde{p}_i)^n = J_i^n$.

In [12]:
c = c_map(m)
ci = inv(c)
r_phasor = ci * r * c
print(r_phasor)

DAMap{StaticArraysCore.MVector{5, ComplexF64}, StaticArraysCore.MVector{6, ComplexTPS64{Descriptor(NV=6, MO=2)}}, Nothing, Nothing}(ComplexF64[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], 6-element StaticArraysCore.MVector{6, ComplexTPS64{Descriptor(NV=6, MO=2)}}:
 Index Real                     Imag                       Order   Exponent
---------------------------------------------------------------------------------------------
   1:   3.2625736656277848e-01  -9.4528097979570247e-01      1      1   0   0   0   0   0
   1:   1.1996841454073626e+00   4.1406290653633482e-01      2      1   0   0   0   0   1
---------------------------------------------------------------------------------------------
   2:   3.2625736656277848e-01   9.4528097979570247e-01      1      0   1   0   0   0   0
   2:   1.1996841454073626e+00  -4.1406290653633482e-01      2      0   1   0   0   0   1
------------------------------------------------------------------------------------------

Note that this map, which is a rotation, is in the phasors basis. So, $\tilde{q}_1 = \sqrt{J}e^{\textrm{i}\phi_1}$. In the linear case we have

$$
\tilde{q}_1 ∘ \vec{r}_{phasor,linear} = \underbrace{\sqrt{J}e^{\textrm{i}\phi_1}}_{\tilde{q}_1}e^{-\textrm{i}2\pi\nu_{1}} = \sqrt{J}e^{\textrm{i}(\phi_1 - 2\pi\nu_{1})} \ .
$$
This makes sense - it rotators phase. In the nonlinear case we see that we have

$$
\tilde{q}_1 ∘ \vec{r}_{phasor} = \sqrt{J}e^{\textrm{i}\phi_1}e^{-\textrm{i}2\pi\left(\nu_{1}+\xi_x\delta\right)}
$$

We can remove the $\sqrt{J}e^{\textrm{i}\phi_1}$ using `par` in `GTPSA`, specifying to remove the first variable. Then we can naturally compute the amplitude-dependent tune

In [13]:
ADT = real(-log(par(r_phasor.v[1], 1))/(2*pi*im))
ADT

TPS64{Descriptor(NV=6, MO=2)}:
 Coefficient                Order   Exponent
  1.9710619563624454e-01      0      0   0   0   0   0   0
 -2.0198826166144121e-01      1      0   0   0   0   0   1


With a differentiable code, we can trivially extract things like the chromaticity, and get exact solutions given the physics of the tracking code - no nasty equations or integrals.

Not only is differentiability necessary to compute and analyze high order Taylor maps, it also allows us to compute exact derivatives of anything we want. This includes the parameters, e.g. the quadrupole strengths. Using `SciBmad`, we can compute the tune $\nu_x$ as a function of variations in the quadrupole strength, e.g. $\nu=\nu(\Delta K_{n1})$